# Исследование надежности заемщиков


Откроем таблицу и изучим общую информацию о данных

Импортируем библиотеку pandas. Прочитаем данные из csv-файла в датафрейм и сохраним в переменную `data`. 


In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:


Выводим первые 20 строчек датафрейма `data` на экран.

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выводим основную информацию о датафрейме с помощью метода `info()`.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выводим количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [7]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выводим медианное значение трудового стажа `days_employed` в днях.**

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

**Выводим перечень уникальных значений столбца `children`.**

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Для проверки ещё раз выводим количество пропущенных значений для каждого столбца с помощью двух методов.**

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

** Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

** Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приводим их к нижнему регистру.**

In [15]:
data['education'] = data['education'].str.lower()

** Выводим на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удаляем их.**

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

** На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

** Выводим на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и отвечаем на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [28]:
data_pivot =  data.pivot_table(index=['children'], values = 'debt', aggfunc = {'sum','count'})
# sum debt - количество клиентов имеющие задержки в оплате по кредитам. len debt - общее количество клиентов
data_pivot['debt'] = data.pivot_table(index=['children'], values = 'debt')
# сводная таблица зависимости кол-ва детей и возврат кредита в срок.
print(data_pivot)

          count   sum      debt
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000


**Вывод:** 
Проанализировав результат можно заключить, что чаще возвращают кредит во время клиенты без детей. Явной зависимости не прослеживается. В целом возврат кредита в срок от числа детей не зависит. Хуже возвращали кредит клиенты с 4 детьми. Клиенты с 5 детьми считаем аномалией. Рекомендуется повторно уточнить информацию, либо не брать в общий анализ.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
data_pivot =  data.pivot_table(index=['family_status'], values = 'debt', aggfunc = {'sum','count'})
# count   - общее количество клиентов, sum  - количество клиентов имеющие задержки в оплате по кредитам. 
data_pivot['debt'] = data.pivot_table(index=['family_status'], values = 'debt')
# сводная таблица зависимости кол-ва детей и возврат кредита в срок.

print(data_pivot)

                       count  sum      debt
family_status                              
Не женат / не замужем   2796  273  0.097639
в разводе               1189   84  0.070648
вдовец / вдова           951   63  0.066246
гражданский брак        4134  385  0.093130
женат / замужем        12261  927  0.075606


**Вывод:** Семейное положение не влияет на возврат кредита в срок. Отметить можно клиентов с семейным статусом **вдовец / вдова**. Доля возвратов кредита в срок у них выше, чем остальных.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
data_pivot =  data.pivot_table(index=['total_income_category'], values = 'debt', aggfunc = {'sum','count'})
# count   - общее количество клиентов, sum  - количество клиентов имеющие задержки в оплате по кредитам. 
data_pivot['debt'] = data.pivot_table(index=['total_income_category'], values = 'debt')
# сводная таблица зависимости кол-ва детей и возврат кредита в срок.

print(data_pivot)

                       count   sum      debt
total_income_category                       
A                         25     2  0.080000
B                       5014   354  0.070602
C                      15921  1353  0.084982
D                        349    21  0.060172
E                         22     2  0.090909


**Вывод:** Предварительно можно сказать, что данных по каждой категории не достаточно для корректного анализа зависимости. 

Отбросив малочисленные категории, можно сделать вывод, что зависимость следующая: чем больше уровень дохода, тем чаще клиенты возвращают кредит в срок.

#### Как разные цели кредита влияют на его возврат в срок?

In [31]:
data_pivot =  data.pivot_table(index=['purpose_category'], values = 'debt', aggfunc = {'sum','count'})
# count   - общее количество клиентов, sum  - количество клиентов имеющие задержки в оплате по кредитам. 
data_pivot['debt'] = data.pivot_table(index=['purpose_category'], values = 'debt')

print(data_pivot)

                          count  sum      debt
purpose_category                              
операции с автомобилем     4279  400  0.093480
операции с недвижимостью  10751  780  0.072551
получение образования      3988  369  0.092528
проведение свадьбы         2313  183  0.079118


**Вывод:** Клиенты с целью кредита "операции с недвижимостью" чаще возвращают кредит в срок. Реже всех возвращают кредит вовремя клиенты с целью "операции с автомобилем".

#### Приведем возможные причины появления пропусков в исходных данных.

*Ответ:* 

- Первая причина - непредоставление информации клиентом (человеческий фактор):

    К примеру: клиент неимеющий ежемесячного дохода или клиент никода не работал(а). Данная информация могла повлиять на получение кредита и клиент мог не указать информацию о себе.


- Вторая причина - произошла техническая ошибка.

    При записи/выгрузки или при подготовке данные могут "потеряться".

#### Объясняем, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медийные значения лучше решение. На них не влияют значения выбивающиеся из общей массы в списке, в отличие от среднего начения.

### Шаг 4: общий вывод.

***Исследование данных и зависимостей***

Иследовав данные получили зависимости показателей "семейное положение", "уровень дохода" и "цели кредита" на возврат кредита в срок, тем самым можно охарактеризовать надежность будущих клиентов.

По зависимости между количеством детей и возвратом кредита в срок можно утверждать, что клиенты без детей чаще возвращают кредиты в срок.

По зависимости разных целей кредита и его возврат в срок - клиенты с большим доходом чаще возвращают кредиты в срок. <br>
Эти зависимости могли оказаться иными, если бы клиентов других категорий было больше.

По зависимости между семейным положением и возвратом кредита в срок можно утверждать что клиенты с семейным положением вдовец / вдова чаще возвращают кредиты в срок.

По зависимости разных целей кредита и его возврат в срок - клиенты с целью кредита "операции с недвижимостью" чаще возвращают кредиты в срок.

***Резумируя можно выделить типы клиентов:***

Клиент надежный: <br>
- Количество детей - без детей
- Семейное положение - вдовец/вдова
- Уровень дохода - не мене 200001, но не более 1000000
- Цель кредита - операции с недвижимостью

Клиент средний: <br>
- Количество детей - от 1 до 2
- Семейное положение - женат/ замужем / в разводе
- Уровень дохода - не мене 50000, но не более 200000
- Цель кредита - проведение свадьбы

Клиент не надежный: <br>
- Количество детей - более 2
- Семейное положение - не женат / не замжем / гражданский брак
- Уровень дохода - до 30000
- Цель кредита - операции с автомобилем / получение образования

***Рекомендации заказчику:***
1. Исключить некорректный ввод данных. 
2. Сделать все поля ввода данных, которые указаны в таблице, обязательными.
3. Повторно предоставить обновленные данные для корректировки анализа.